d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# 2.5 Spark UI

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this notebook you:<br>
* Analyze the Spark UI
* Understand how distributed operations are implemented

In [0]:
%run ../Includes/Classroom-Setup

Data mounted to /mnt/davis ...

## Count

Let's start with a simple example of analyzing how `count()` is implemented.

In [0]:
%sql
SELECT count(*) FROM fireCalls

count(1)
240613


-sandbox
## Stage Boundaries

Start by expanding the Spark Job. You'll notice the job was broken into two stages. 

<div><img src="http://files.training.databricks.com/images/eLearning/ucdavis/countfireCalls2.png" style="height: 300px; border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa; margin: 20px"/></div>

When we perform a count, each executor has to sum up their count locally. Only once all of them finish, then one slot is tasked with adding up all of the counts from the other executors. So it doesn't matter if you have one fast executor at counting - the bottleneck is the slowest executor :)

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> The Community Edition version of Databricks now provisions a single driver node with 2 cores for you.  This change explains why your output might look different from the video.

-sandbox
## Shuffle Write

When our executors locally count their number of records, they have to write it to their local disk for that one designated executor to read. This causes a "shuffle write". Let's see that in the Spark UI. Click on the `i` icon next to the first stage.

<div><br><img src="http://files.training.databricks.com/images/eLearning/ucdavis/shuffleWrite.png" style="height: 300px; border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa; margin: 20px"/></div>

You'll notice the **2 shuffle records** that were written out (each of them is very small).

-sandbox
## Shuffle Read

When that designated executor reads in the "shuffle files" from the other executors that is called a "shuffle read". Click on the `i` icon next to the second stage.

<div><br><img src="http://files.training.databricks.com/images/eLearning/ucdavis/shuffleRead.png" style="height: 300px; border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa; margin: 20px"/></div>

You'll notice the **2 shuffle records** that were read in by one slot.

## Examine the Spark UI

In [0]:
%sql
SELECT `call type`, count(*) AS count
FROM fireCalls
GROUP BY `call type`
ORDER BY count DESC

call type,count
Medical Incident,156374
Structure Fire,31329
Alarms,26090
Traffic Collision,9749
Other,3799
Citizen Assist / Service Call,3600
Outside Fire,2940
Vehicle Fire,1101
Water Rescue,1096
Gas Leak (Natural and LP Gases),888


Do you recall that 200/200 parameter? Because of it, we have to have 200 resulting partitions, but the record count for each of those partitions is very small! Let's re-run this query but reduce the `spark.sql.shuffle.partitions` to 8.

In [0]:
%sql
SET spark.sql.shuffle.partitions=8

key,value
spark.sql.shuffle.partitions,8


-sandbox
## Skewed Partitions

If you look at the Spark UI, you'll notice there is significant skew in our data. Most partitions don't actually have any records!

<div><br><img src="https://files.training.databricks.com/images/eLearning/ucdavis/skewedRead.png" style="height: 300px; border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa; margin: 20px"/></div>

## Re-run Query

Let's try re-running that query.

In [0]:
%sql
SELECT `call type`, count(*) AS count
FROM fireCalls
GROUP BY `call type`
ORDER BY count DESC

call type,count
Medical Incident,156374
Structure Fire,31329
Alarms,26090
Traffic Collision,9749
Other,3799
Citizen Assist / Service Call,3600
Outside Fire,2940
Vehicle Fire,1101
Water Rescue,1096
Gas Leak (Natural and LP Gases),888


-sandbox
Great! You'll notice that the resulting partitions have a greater number of records and there is less skew across partitions.

<div><br><img src="http://files.training.databricks.com/images/eLearning/ucdavis/lessSkewedRead.png" style="height: 300px; border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa; margin: 20px"/></div>

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>